In [1]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score
from model_build.kmeans_simple import bad_games
import django
from django.db.models import Q, Case, When, IntegerField
from django.db.models import F as dF
%matplotlib inline
from matplotlib import pyplot as plt
import scipy.stats as st

django.setup()

from nbad3.models import Coords, Moment, Possession
from datetime import timedelta
import numpy as np
from ingest.utils import ball_team
import argparse
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.metrics import silhouette_score
import os
from scipy import stats
from itertools import combinations
import sys
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
from IPython.display import display
import torch
from torch.utils.data import DataLoader
from model_build.build_utils import OnLoadPossessionsDataset
from model_build.nets import EncoderCNN, DecoderRNN

from tslearn.clustering import TimeSeriesKMeans
from tslearn.metrics import dtw
from tslearn.generators import random_walks

random_state = 0

bad_poss_ids = [27879, 34842, 28986, 49167, 49992, 51386, 58341]

In [2]:
arr = np.load('/dbdata/all_good_out.npy')
arr.shape

(7931, 1710)

In [3]:
hc_possessions = Possession.objects.filter(valid=True, half_court=True).order_by('id')
hc_possessions = hc_possessions.exclude(game__in=bad_games)
hc_possessions = hc_possessions.exclude(id__in=bad_poss_ids)

In [6]:
poss_clusters = list(zip([poss.id for poss in hc_possessions], [poss.long_desc() for poss in hc_possessions], [poss.points for poss in hc_possessions]))

In [7]:
df = pd.DataFrame(poss_clusters, columns=["poss_id", "poss_desc", "points"])
df.head(5)

,poss_id,poss_desc,points
0,18852,"Possession for team Charlotte Hornets (1610612766) from game Charlotte Hornets @ Toronto Raptors, 01/01/2016, scored 2 points; Start event: Valanciunas Putback Dunk (2 PTS) with 0:11:35 left in period 1 (#4); End event: Zeller 1' Dunk (2 PTS) (Batum 1 AST) with 0:11:25 left in period 1 (#5)",2
1,18853,"Possession for team Toronto Raptors (1610612761) from game Charlotte Hornets @ Toronto Raptors, 01/01/2016, scored 0 points; Start event: Zeller 1' Dunk (2 PTS) (Batum 1 AST) with 0:11:25 left in period 1 (#5); End event: Williams REBOUND (Off:0 Def:1) with 0:11:06 left in period 1 (#7)",0
2,18857,"Possession for team Toronto Raptors (1610612761) from game Charlotte Hornets @ Toronto Raptors, 01/01/2016, scored 0 points; Start event: Scola REBOUND (Off:0 Def:1) with 0:10:18 left in period 1 (#17); End event: Zeller REBOUND (Off:0 Def:1) with 0:09:49 left in period 1 (#23)",0
3,18861,"Possession for team Toronto Raptors (1610612761) from game Charlotte Hornets @ Toronto Raptors, 01/01/2016, scored 3 points; Start event: Carroll REBOUND (Off:0 Def:1) with 0:09:26 left in period 1 (#27); End event: DeRozan 24' 3PT Jump Shot (3 PTS) (Lowry 1 AST) with 0:09:13 left in period 1 (#28)",3
4,18862,"Possession for team Charlotte Hornets (1610612766) from game Charlotte Hornets @ Toronto Raptors, 01/01/2016, scored 2 points; Start event: DeRozan 24' 3PT Jump Shot (3 PTS) (Lowry 1 AST) with 0:09:13 left in period 1 (#28); End event: Hairston 20' Pullup Jump Shot (2 PTS) with 0:08:54 left in period 1 (#29)",2


In [62]:
def conf_int(a):
    return st.t.interval(0.95, len(a)-1, loc=0, scale=st.sem(a))

In [85]:
ts_arr = arr.reshape(7931, 30, -1)

In [86]:
clusterer = TimeSeriesKMeans(n_clusters=14, metric='dtw')

In [87]:
outputs = clusterer.fit_predict(ts_arr)

2429893.723 --> 1862369.620 --> 1822638.127 --> 1806416.770 --> 1797817.270 --> 1791665.690 --> 1786766.277 --> 1783239.599 --> 1780130.675 --> 1777876.689 --> 1776119.630 --> 1774533.119 --> 1773284.933 --> 1772192.306 --> 1771210.322 --> 1770187.234 --> 1768955.958 --> 1767268.781 --> 1764899.802 --> 1762583.413 --> 1760460.443 --> 1758730.288 --> 1757292.668 --> 1755964.308 --> 1754939.385 --> 1754125.604 --> 1753593.911 --> 1753273.400 --> 1753097.658 --> 1752981.394 --> 1752904.646 --> 1752838.465 --> 1752760.195 --> 1752696.708 --> 1752626.534 --> 1752546.614 --> 1752453.790 --> 1752341.171 --> 1752291.686 --> 1752231.946 --> 1752176.559 --> 1752105.713 --> 1752021.089 --> 1751936.542 --> 1751853.051 --> 1751766.148 --> 1751711.300 --> 1751658.118 --> 1751611.348 --> 1751546.479 --> 


In [88]:
df['ts_clusters'] = outputs

In [89]:
outputs.shape

(7931,)

In [96]:
centers = dict(enumerate(clusterer.cluster_centers_))
cent_dists = []
for label, data, desc in zip(couts, ts_arr, [poss.long_desc() for poss in hc_possessions]):
    cent_dists.append(dtw(data, centers[label]))
df['new_cent_dists'] = cent_dists

In [97]:
for clust in range(14):
    display(df[df.ts_clusters == clust].sort_values(by='new_cent_dists', ascending=True).head(5))

,poss_id,poss_desc,cluster,points,cent_dist,newclust1,new_cent_dists
1154,24824,"Possession for team Los Angeles Lakers (1610612747) from game Golden State Warriors @ Los Angeles Lakers, 01/05/2016, scored 0 points; Start event: Nance Jr. REBOUND (Off:0 Def:3) with 0:03:41 left in period 2 (#227); End event: Barnes REBOUND (Off:1 Def:2) with 0:03:20 left in period 2 (#229)",10,0,1250.256337,0,1222.076620
7032,59399,"Possession for team Cleveland Cavaliers (1610612739) from game New York Knicks @ Cleveland Cavaliers, 11/04/2015, scored 2 points; Start event: Porzingis 2' Finger Roll Layup (2 PTS) (Anthony 1 AST) with 0:10:17 left in period 1 (#16); End event: Love 16' Jump Shot (2 PTS) with 0:09:56 left in period 1 (#17)",10,2,1261.347293,0,1273.545781
6354,56078,"Possession for team Boston Celtics (1610612738) from game San Antonio Spurs @ Boston Celtics, 11/01/2015, scored 0 points; Start event: Bradley REBOUND (Off:0 Def:2) with 0:09:55 left in period 1 (#15); End event: Spurs Rebound with 0:09:44 left in period 1 (#17)",10,0,1145.150738,0,1308.497491
1406,25852,"Possession for team Cleveland Cavaliers (1610612739) from game Cleveland Cavaliers @ Washington Wizards, 01/06/2016, scored 0 points; Start event: Temple Free Throw 1 of 1 (8 PTS) with 0:08:06 left in period 3 (#337); End event: Porter Jr. REBOUND (Off:2 Def:3) with 0:07:43 left in period 3 (#340)",10,0,1314.148985,0,1327.270697
2769,32908,"Possession for team Los Angeles Clippers (1610612746) from game New Orleans Pelicans @ Los Angeles Clippers, 01/10/2016, scored 2 points; Start event: Pierce REBOUND (Off:1 Def:4) with 0:06:46 left in period 3 (#343); End event: Mbah a Moute 3' Cutting Layup Shot (10 PTS) (Paul 7 AST) with 0:06:29 left in period 3 (#344)",6,2,1275.343599,0,1328.370981


,poss_id,poss_desc,cluster,points,cent_dist,newclust1,new_cent_dists
5477,49915,"Possession for team Portland Trail Blazers (1610612757) from game New Orleans Pelicans @ Portland Trail Blazers, 10/28/2015, scored 0 points; Start event: Lillard REBOUND (Off:0 Def:3) with 0:11:05 left in period 3 (#330); End event: Cunningham REBOUND (Off:0 Def:2) with 0:10:51 left in period 3 (#332)",8,0,767.466218,1,640.616846
1579,26622,"Possession for team Indiana Pacers (1610612754) from game Indiana Pacers @ Orlando Magic, 01/06/2016, scored 2 points; Start event: Fournier 23' Jump Shot (8 PTS) (Hezonja 2 AST) with 0:11:18 left in period 4 (#364); End event: Hill 9' Hook Shot (4 PTS) with 0:10:55 left in period 4 (#367)",8,2,785.721076,1,640.889819
7230,60116,"Possession for team Portland Trail Blazers (1610612757) from game Portland Trail Blazers @ Utah Jazz, 11/04/2015, scored 2 points; Start event: Hayward Free Throw 2 of 2 (16 PTS) with 0:11:00 left in period 4 (#448); End event: Harkless 2' Layup (10 PTS) with 0:10:38 left in period 4 (#450)",8,2,818.840516,1,671.745013
5600,50705,"Possession for team Utah Jazz (1610612762) from game Utah Jazz @ Detroit Pistons, 10/28/2015, scored 0 points; Start event: Jackson 5' Driving Bank Shot (10 PTS) with 0:10:59 left in period 3 (#227); End event: Caldwell-Pope REBOUND (Off:0 Def:3) with 0:10:34 left in period 3 (#229)",8,0,791.097318,1,678.017989
4271,41274,"Possession for team Portland Trail Blazers (1610612757) from game Portland Trail Blazers @ Washington Wizards, 01/18/2016, scored 0 points; Start event: Nene 2' Alley Oop Layup (8 PTS) (Sessions 1 AST) with 0:09:02 left in period 2 (#139); End event: Sessions REBOUND (Off:0 Def:1) with 0:08:42 left in period 2 (#141)",8,0,747.102120,1,678.946162


,poss_id,poss_desc,cluster,points,cent_dist,newclust1,new_cent_dists
4022,40066,"Possession for team Chicago Bulls (1610612741) from game Chicago Bulls @ Detroit Pistons, 01/18/2016, scored 2 points; Start event: Portis REBOUND (Off:1 Def:1) with 0:09:35 left in period 2 (#114); End event: Brooks 1' Driving Layup (4 PTS) with 0:09:14 left in period 2 (#115)",4,2,1271.965373,2,1524.211748
2052,29753,"Possession for team Oklahoma City Thunder (1610612760) from game Oklahoma City Thunder @ Los Angeles Lakers, 01/08/2016, scored 0 points; Start event: Hibbert Putback Dunk (2 PTS) with 0:09:08 left in period 1 (#31); End event: Nance Jr. STEAL (1 STL), Durant Bad Pass Turnover (P1.T2) with 0:08:55 left in period 1 (#33)",4,0,980.022549,2,1540.868415
2160,30038,"Possession for team Orlando Magic (1610612753) from game Orlando Magic @ Brooklyn Nets, 01/08/2016, scored 0 points; Start event: Johnson 12' Pullup Jump Shot (10 PTS) with 0:07:21 left in period 3 (#269); End event: Larkin REBOUND (Off:0 Def:3) with 0:06:43 left in period 3 (#273)",4,0,1423.079247,2,1623.277770
4869,45321,"Possession for team Boston Celtics (1610612738) from game Chicago Bulls @ Boston Celtics, 01/22/2016, scored 3 points; Start event: Zeller REBOUND (Off:1 Def:2) with 0:10:48 left in period 4 (#451); End event: Zeller Free Throw 2 of 2 (5 PTS) with 0:10:26 left in period 4 (#458)",4,3,1336.931753,2,1722.056219
5213,47358,"Possession for team Indiana Pacers (1610612754) from game Indiana Pacers @ Sacramento Kings, 01/23/2016, scored 1 points; Start event: Ellis REBOUND (Off:0 Def:2) with 0:02:05 left in period 3 (#364); End event: Koufos REBOUND (Off:0 Def:2) with 0:01:44 left in period 3 (#371)",4,1,1281.776026,2,1736.557081


,poss_id,poss_desc,cluster,points,cent_dist,newclust1,new_cent_dists
7354,60872,"Possession for team Charlotte Hornets (1610612766) from game Charlotte Hornets @ Dallas Mavericks, 11/05/2015, scored 2 points; Start event: Williams REBOUND (Off:2 Def:8) with 0:08:25 left in period 3 (#287); End event: Jefferson 13' Turnaround Hook Shot (19 PTS) with 0:08:03 left in period 3 (#288)",11,2,1051.522280,3,1447.252933
7406,60984,"Possession for team Portland Trail Blazers (1610612757) from game Memphis Grizzlies @ Portland Trail Blazers, 11/05/2015, scored 0 points; Start event: Vonleh REBOUND (Off:0 Def:1) with 0:05:26 left in period 1 (#72); End event: Allen REBOUND (Off:0 Def:1) with 0:05:12 left in period 1 (#74)",0,0,916.086480,3,1497.781616
5163,47199,"Possession for team Denver Nuggets (1610612743) from game Detroit Pistons @ Denver Nuggets, 01/23/2016, scored 2 points; Start event: Baynes Running Dunk (11 PTS) (Johnson 1 AST) with 0:07:01 left in period 4 (#449); End event: Gallinari Free Throw 2 of 2 (25 PTS) with 0:06:36 left in period 4 (#454)",11,2,921.424913,3,1522.581337
64,19009,"Possession for team Toronto Raptors (1610612761) from game Charlotte Hornets @ Toronto Raptors, 01/01/2016, scored 0 points; Start event: Williams 25' 3PT Jump Shot (14 PTS) (Batum 6 AST) with 0:05:15 left in period 4 (#486); End event: Batum REBOUND (Off:0 Def:3) with 0:04:54 left in period 4 (#488)",11,0,854.562919,3,1539.948528
3143,35926,"Possession for team Golden State Warriors (1610612744) from game Golden State Warriors @ Denver Nuggets, 01/13/2016, scored 0 points; Start event: Gallinari Free Throw 2 of 2 (8 PTS) with 0:02:43 left in period 1 (#114); End event: Gallinari REBOUND (Off:0 Def:2) with 0:02:28 left in period 1 (#117)",0,0,1287.840976,3,1558.290715


,poss_id,poss_desc,cluster,points,cent_dist,newclust1,new_cent_dists
386,20700,"Possession for team Utah Jazz (1610612762) from game Memphis Grizzlies @ Utah Jazz, 01/02/2016, scored 2 points; Start event: Conley Free Throw 1 of 1 (15 PTS) with 0:06:50 left in period 4 (#425); End event: Hood 8' Pullup Jump Shot (22 PTS) with 0:06:33 left in period 4 (#427)",6,2,1046.511339,4,1310.987941
3465,37273,"Possession for team New York Knicks (1610612752) from game New York Knicks @ Brooklyn Nets, 01/13/2016, scored 2 points; Start event: Lopez Free Throw 2 of 2 (18 PTS) with 0:03:20 left in period 4 (#429); End event: Afflalo 17' Pullup Jump Shot (10 PTS) with 0:02:59 left in period 4 (#431)",6,2,1267.213424,4,1421.848165
2385,31104,"Possession for team Miami Heat (1610612748) from game Miami Heat @ Utah Jazz, 01/09/2016, scored 0 points; Start event: Gobert Dunk (2 PTS) with 0:11:01 left in period 1 (#6); End event: Lyles REBOUND (Off:0 Def:1) with 0:10:31 left in period 1 (#8)",6,0,1091.124362,4,1460.048247
7511,61399,"Possession for team Chicago Bulls (1610612741) from game Oklahoma City Thunder @ Chicago Bulls, 11/05/2015, scored 2 points; Start event: Butler REBOUND (Off:1 Def:2) with 0:07:37 left in period 2 (#185); End event: Gibson Cutting Dunk Shot (5 PTS) (Rose 2 AST) with 0:07:21 left in period 2 (#186)",6,2,1181.036444,4,1482.730408
124,19711,"Possession for team Philadelphia 76ers (1610612755) from game Philadelphia 76ers @ Los Angeles Lakers, 01/01/2016, scored 0 points; Start event: Covington REBOUND (Off:0 Def:5) with 0:03:32 left in period 3 (#401); End event: Brown REBOUND (Off:0 Def:5) with 0:03:10 left in period 3 (#403)",6,0,1041.089475,4,1484.560819


...

,poss_id,poss_desc,cluster,points,cent_dist,newclust1,new_cent_dists
2270,30547,"Possession for team Charlotte Hornets (1610612766) from game Charlotte Hornets @ Los Angeles Clippers, 01/09/2016, scored 2 points; Start event: Crawford 18' Jump Shot (8 PTS) with 0:11:17 left in period 2 (#134); End event: Lamb 2' Driving Finger Roll Layup (5 PTS) (Lin 2 AST) with 0:10:57 left in period 2 (#135)",6,2,1277.094648,9,1463.011056
3735,38389,"Possession for team Cleveland Cavaliers (1610612739) from game Cleveland Cavaliers @ Houston Rockets, 01/15/2016, scored 0 points; Start event: Howard 8' Hook Shot (12 PTS) (Thornton 1 AST) with 0:09:03 left in period 3 (#337); End event: Ariza REBOUND (Off:0 Def:4) with 0:08:39 left in period 3 (#341)",5,0,1425.474650,9,1514.575595
469,21153,"Possession for team Oklahoma City Thunder (1610612760) from game Oklahoma City Thunder @ Charlotte Hornets, 01/02/2016, scored 2 points; Start event: Lin 20' Jump Shot (2 PTS) (Williams 1 AST) with 0:06:36 left in period 1 (#46); End event: Adams Dunk (4 PTS) (Durant 1 AST) with 0:06:20 left in period 1 (#47)",7,2,905.432405,9,1527.987764
4380,42206,"Possession for team Oklahoma City Thunder (1610612760) from game Oklahoma City Thunder @ Denver Nuggets, 01/19/2016, scored 2 points; Start event: Gallinari 24' 3PT Jump Shot (3 PTS) (Mudiay 1 AST) with 0:10:39 left in period 1 (#13); End event: Durant 21' Jump Shot (2 PTS) (Roberson 1 AST) with 0:10:27 left in period 1 (#14)",7,2,1133.670359,9,1622.046193
2439,31226,"Possession for team Utah Jazz (1610612762) from game Miami Heat @ Utah Jazz, 01/09/2016, scored 0 points; Start event: Ingles REBOUND (Off:0 Def:2) with 0:02:26 left in period 3 (#323); End event: Hayward Lost Ball Turnover (P7.T15), Bosh STEAL (1 STL) with 0:02:09 left in period 3 (#324)",7,0,1373.241349,9,1646.418471


,poss_id,poss_desc,cluster,points,cent_dist,newclust1,new_cent_dists
1606,26686,"Possession for team Portland Trail Blazers (1610612757) from game Los Angeles Clippers @ Portland Trail Blazers, 01/06/2016, scored 2 points; Start event: Jordan Running Alley Oop Dunk Shot (6 PTS) (Paul 4 AST) with 0:05:58 left in period 1 (#50); End event: Plumlee 5' Turnaround Hook Shot (4 PTS) with 0:05:31 left in period 1 (#51)",5,2,998.386360,10,1320.010899
7443,61053,"Possession for team Portland Trail Blazers (1610612757) from game Memphis Grizzlies @ Portland Trail Blazers, 11/05/2015, scored 3 points; Start event: Lee 21' Jump Shot (14 PTS) with 0:11:15 left in period 3 (#241); End event: Leonard Free Throw 3 of 3 (12 PTS) with 0:11:00 left in period 3 (#248)",5,3,1312.253316,10,1351.638042
7614,62101,"Possession for team Denver Nuggets (1610612743) from game Denver Nuggets @ Golden State Warriors, 11/06/2015, scored 0 points; Start event: Nelson REBOUND (Off:0 Def:1) with 0:05:50 left in period 4 (#469); End event: Ezeli STEAL (1 STL), Foye Bad Pass Turnover (P2.T16) with 0:05:30 left in period 4 (#480)",5,0,1068.329400,10,1389.124124
1749,27251,"Possession for team Brooklyn Nets (1610612751) from game Toronto Raptors @ Brooklyn Nets, 01/06/2016, scored 0 points; Start event: Larkin REBOUND (Off:0 Def:2) with 0:02:24 left in period 1 (#84); End event: Valanciunas REBOUND (Off:2 Def:2) with 0:02:02 left in period 1 (#86)",5,0,1227.916757,10,1471.316040
2092,29851,"Possession for team Los Angeles Lakers (1610612747) from game Oklahoma City Thunder @ Los Angeles Lakers, 01/08/2016, scored 2 points; Start event: Westbrook 15' Pullup Jump Shot (19 PTS) with 0:08:57 left in period 3 (#287); End event: Bryant 4' Driving Hook Shot (13 PTS) with 0:08:31 left in period 3 (#288)",2,2,1144.327934,10,1482.307913


,poss_id,poss_desc,cluster,points,cent_dist,newclust1,new_cent_dists
4857,45183,"Possession for team Boston Celtics (1610612738) from game Chicago Bulls @ Boston Celtics, 01/22/2016, scored 0 points; Start event: Olynyk REBOUND (Off:0 Def:2) with 0:06:00 left in period 1 (#64); End event: Butler REBOUND (Off:1 Def:3) with 0:05:51 left in period 1 (#66)",9,0,1158.603624,11,1159.040469
3277,36486,"Possession for team Miami Heat (1610612748) from game Miami Heat @ Los Angeles Clippers, 01/13/2016, scored 3 points; Start event: Rivers 2' Driving Layup (8 PTS) with 0:01:35 left in period 4 (#523); End event: Andersen 26' 3PT Jump Shot (3 PTS) (Richardson 1 AST) with 0:01:20 left in period 4 (#524)",9,3,1298.679206,11,1296.838279
4981,45909,"Possession for team Toronto Raptors (1610612761) from game Miami Heat @ Toronto Raptors, 01/22/2016, scored 0 points; Start event: Wade 3' Driving Layup (18 PTS) (Winslow 1 AST) with 0:07:24 left in period 4 (#439); End event: Scola Traveling Turnover (P1.T14) with 0:07:05 left in period 4 (#440)",9,0,1287.192478,11,1517.429260
6363,56209,"Possession for team San Antonio Spurs (1610612759) from game San Antonio Spurs @ Boston Celtics, 11/01/2015, scored 2 points; Start event: Bradley 8' Floating Jump Shot (11 PTS) with 0:03:52 left in period 3 (#387); End event: Aldridge Free Throw 2 of 2 (16 PTS) with 0:03:36 left in period 3 (#391)",9,2,1596.946378,11,1601.317300
7852,63074,"Possession for team New York Knicks (1610612752) from game Milwaukee Bucks @ New York Knicks, 11/06/2015, scored 0 points; Start event: Henson Free Throw 2 of 2 (13 PTS) with 0:10:07 left in period 4 (#461); End event: Copeland REBOUND (Off:0 Def:1) with 0:09:49 left in period 4 (#464)",9,0,1146.266834,11,1679.896560


,poss_id,poss_desc,cluster,points,cent_dist,newclust1,new_cent_dists
529,21334,"Possession for team Orlando Magic (1610612753) from game Orlando Magic @ Cleveland Cavaliers, 01/02/2016, scored 0 points; Start event: Irving 1' Driving Layup (2 PTS) (James 1 AST) with 0:08:18 left in period 1 (#26); End event: Thompson REBOUND (Off:1 Def:2) with 0:07:56 left in period 1 (#28)",3,0,1045.595281,12,1020.650280
3228,36376,"Possession for team Miami Heat (1610612748) from game Miami Heat @ Los Angeles Clippers, 01/13/2016, scored 3 points; Start event: Redick 19' Jump Shot (7 PTS) (Aldrich 1 AST) with 0:05:01 left in period 2 (#194); End event: Udrih 29' 3PT Jump Shot (8 PTS) (Bosh 4 AST) with 0:04:42 left in period 2 (#195)",3,3,1094.114002,12,1047.571798
257,20331,"Possession for team Detroit Pistons (1610612765) from game Detroit Pistons @ Indiana Pacers, 01/02/2016, scored 2 points; Start event: Miles 16' Jump Shot (6 PTS) with 0:10:40 left in period 4 (#396); End event: Johnson 1' Driving Layup (3 PTS) with 0:10:20 left in period 4 (#397)",12,2,1037.299484,12,1056.521815
1229,25394,"Possession for team Dallas Mavericks (1610612742) from game Sacramento Kings @ Dallas Mavericks, 01/05/2016, scored 2 points; Start event: Felton REBOUND (Off:0 Def:4) with 0:06:23 left in period 2 (#187); End event: Harris 18' Pullup Jump Shot (2 PTS) with 0:06:02 left in period 2 (#188)",3,2,1145.243040,12,1117.419100
1656,26784,"Possession for team Los Angeles Clippers (1610612746) from game Los Angeles Clippers @ Portland Trail Blazers, 01/06/2016, scored 2 points; Start event: Plumlee 14' Jump Shot (15 PTS) with 0:02:52 left in period 3 (#344); End event: Jordan Alley Oop Dunk (9 PTS) (Paul 12 AST) with 0:02:34 left in period 3 (#345)",12,2,1152.696953,12,1162.720347


,poss_id,poss_desc,cluster,points,cent_dist,newclust1,new_cent_dists
5180,47264,"Possession for team Indiana Pacers (1610612754) from game Indiana Pacers @ Sacramento Kings, 01/23/2016, scored 3 points; Start event: Koufos 3' Layup (2 PTS) (Cauley-Stein 1 AST) with 0:02:25 left in period 1 (#96); End event: Ellis 3PT Jump Shot (10 PTS) (G. Hill 2 AST) with 0:01:58 left in period 1 (#97)",1,3,754.654719,13,938.683470
2243,30291,"Possession for team Washington Wizards (1610612764) from game Toronto Raptors @ Washington Wizards, 01/08/2016, scored 0 points; Start event: DeRozan Free Throw 2 of 2 (31 PTS) with 0:03:47 left in period 4 (#496); End event: Wall Out of Bounds - Bad Pass Turnover Turnover (P7.T19) with 0:03:32 left in period 4 (#499)",1,0,800.222064,13,1041.245073
6461,56514,"Possession for team Memphis Grizzlies (1610612763) from game Memphis Grizzlies @ Golden State Warriors, 11/02/2015, scored 0 points; Start event: Wright REBOUND (Off:0 Def:1) with 0:11:35 left in period 2 (#132); End event: Speights REBOUND (Off:0 Def:2) with 0:11:20 left in period 2 (#134)",1,0,960.615890,13,1066.227002
4505,43267,"Possession for team Houston Rockets (1610612745) from game Detroit Pistons @ Houston Rockets, 01/20/2016, scored 0 points; Start event: Ilyasova 7' Fadeaway Bank Shot (8 PTS) (Jackson 5 AST) with 0:02:58 left in period 2 (#262); End event: Baynes REBOUND (Off:1 Def:1) with 0:02:34 left in period 2 (#264)",1,0,948.667597,13,1072.078526
1859,27990,"Possession for team Boston Celtics (1610612738) from game Boston Celtics @ Chicago Bulls, 01/07/2016, scored 2 points; Start event: Rose 1' Layup (18 PTS) (Gasol 4 AST) with 0:03:19 left in period 4 (#492); End event: Olynyk 1' Driving Layup (16 PTS) with 0:03:01 left in period 4 (#493)",1,2,887.708190,13,1106.517036


In [98]:
agged = df.groupby('ts_clusters').agg({'new_cent_dists': ('mean', 'median'), 'points': ('count', 'mean', 'std')}).sort_values(by=('points', 'mean'), ascending=False)
agged.to_csv('tsagged.csv')
agged

new_cent_dists              points                    
                    mean       median  count      mean       std
newclust1                                                       
3          2369.716746    2383.256746  726    1.112948  1.160700
2          2635.909053    2650.844797  486    1.100823  1.154158
12         2545.587084    2610.600019  466    1.090129  1.138018
4          2313.739669    2301.056708  433    1.087760  1.139226
0          2742.354654    2768.922947  528    1.039773  1.141061
6          2086.587799    2049.994727  572    1.034965  1.134787
8          2442.775128    2474.775662  547    1.034735  1.129313
9          2338.173621    2321.220337  438    1.029680  1.097755
7          2147.846801    2214.827714  616    1.027597  1.150843
13         2601.777584    2801.346990  628    1.027070  1.146759
10         2518.727865    2554.140735  643    0.989114  1.128037
1          1715.661242    1713.505667  636    0.984277  1.137647
5          2419.202644    2476.936242  575    0.958261  1.148396
11         2748.367424    2767.871604  637    0.946625  1.121674

In [100]:
wizards = df[df.poss_desc.str.contains("for team Washington")]
wizards

,poss_id,poss_desc,cluster,points,cent_dist,newclust1,new_cent_dists
656,22306,"Possession for team Washington Wizards (1610612764) from game Miami Heat @ Washington Wizards, 01/03/2016, scored 0 points; Start event: Dudley REBOUND (Off:0 Def:2) with 0:10:13 left in period 1 (#18); End event: Deng REBOUND (Off:0 Def:1) with 0:09:59 left in period 1 (#22)",8,0,1191.406140,1,2603.727415
657,22308,"Possession for team Washington Wizards (1610612764) from game Miami Heat @ Washington Wizards, 01/03/2016, scored 0 points; Start event: Bosh 17' Jump Shot (2 PTS) (Dragic 1 AST) with 0:09:52 left in period 1 (#23); End event: Whiteside REBOUND (Off:0 Def:3) with 0:09:28 left in period 1 (#28)",9,0,1092.602985,11,2722.478093
658,22312,"Possession for team Washington Wizards (1610612764) from game Miami Heat @ Washington Wizards, 01/03/2016, scored 0 points; Start event: Deng 3' Layup (2 PTS) (Bosh 1 AST) with 0:08:45 left in period 1 (#42); End event: Temple Bad Pass Turnover (P1.T2), Dragic STEAL (1 STL) with 0:08:30 left in period 1 (#43)",4,0,1432.633078,2,2948.451729
659,22314,"Possession for team Washington Wizards (1610612764) from game Miami Heat @ Washington Wizards, 01/03/2016, scored 2 points; Start event: Dragic 2' Running Layup (2 PTS) with 0:08:27 left in period 1 (#44); End event: Wall 21' Pullup Jump Shot (2 PTS) with 0:08:09 left in period 1 (#45)",10,2,1367.277550,0,2379.008624
665,22328,"Possession for team Washington Wizards (1610612764) from game Miami Heat @ Washington Wizards, 01/03/2016, scored 2 points; Start event: Gortat REBOUND (Off:3 Def:2) with 0:04:28 left in period 1 (#72); End event: Gortat 1' Putback Layup (2 PTS) with 0:04:03 left in period 1 (#75)",9,2,871.131682,11,3041.398035
668,22334,"Possession for team Washington Wizards (1610612764) from game Miami Heat @ Washington Wizards, 01/03/2016, scored 0 points; Start event: Johnson 3PT Jump Shot (3 PTS) (Deng 1 AST) with 0:02:57 left in period 1 (#86); End event: Green REBOUND (Off:0 Def:1) with 0:02:43 left in period 1 (#88)",9,0,1010.828813,11,3256.380822
670,22340,"Possession for team Washington Wizards (1610612764) from game Miami Heat @ Washington Wizards, 01/03/2016, scored 0 points; Start event: Bosh Free Throw 2 of 2 (8 PTS) with 0:01:40 left in period 1 (#111); End event: Johnson REBOUND (Off:0 Def:1) with 0:01:14 left in period 1 (#115)",8,0,1434.002782,8,2765.083718
672,22350,"Possession for team Washington Wizards (1610612764) from game Miami Heat @ Washington Wizards, 01/03/2016, scored 0 points; Start event: Bosh 27' 3PT Jump Shot (13 PTS) (Udrih 2 AST) with 0:11:37 left in period 2 (#141); End event: Bosh REBOUND (Off:0 Def:1) with 0:11:16 left in period 2 (#143)",5,0,1431.867061,9,2275.015351
675,22354,"Possession for team Washington Wizards (1610612764) from game Miami Heat @ Washington Wizards, 01/03/2016, scored 0 points; Start event: Sessions REBOUND (Off:0 Def:1) with 0:10:26 left in period 2 (#159); End event: Sessions Lost Ball Turnover (P1.T4), Winslow STEAL (1 STL) with 0:10:11 left in period 2 (#160)",6,0,2036.270741,4,2674.410699
677,22360,"Possession for team Washington Wizards (1610612764) from game Miami Heat @ Washington Wizards, 01/03/2016, scored 0 points; Start event: WIZARDS Rebound with 0:08:52 left in period 2 (#171); End event: Winslow REBOUND (Off:0 Def:1) with 0:08:37 left in period 2 (#175)",0,0,902.103896,5,2189.167663


In [101]:
wiz_agged = wizards.groupby('cluster').agg({'cent_dist': ('mean', 'median'), 'points': ('count', 'mean', 'std')}).sort_values(by=('points', 'mean'), ascending=False)
wiz_agged.to_csv('wiz_agged.csv')
wiz_agged

points                        cent_dist             
         count      mean       std         mean       median
cluster                                                     
11       8      2.000000  0.925820  1387.482161  1408.145412
12       10     1.400000  1.074968  1334.415778  1288.657159
7        24     1.208333  1.284664  1310.414706  1264.352893
0        24     1.125000  1.190999  1166.348327  1053.716640
4        16     1.062500  1.123610  1443.612059  1270.205236
5        30     1.033333  1.188547  1234.453166  1187.634742
6        7      1.000000  1.290994  1520.054702  1439.953492
9        25     1.000000  1.190238  1144.113584  1092.602985
1        27     0.962963  1.055443  1129.439317  1045.510447
10       19     0.947368  1.177270  1358.039198  1344.258699
8        15     0.933333  0.961150  1318.659567  1199.444011
3        12     0.750000  1.138180  1215.511275  1114.727841
2        3      0.666667  1.154701  1190.513919  1252.271947
13       12     0.333333  0.887625  1283.264575  1334.736750

In [110]:
for cluster in [3, 2, 12, 4, 0, 6, 8, 9, 7, 13, 10, 1, 5, 11]:
    pts = wizards[wizards.cluster == cluster].points
    print(str(round(pts.mean(), 3)) + " +/- " + str(round(conf_int(pts)[1], 3)))

0.75 +/- 0.723
0.667 +/- 2.868
1.4 +/- 0.769
1.062 +/- 0.599
1.125 +/- 0.503
1.0 +/- 1.194
0.933 +/- 0.532
1.0 +/- 0.491
1.208 +/- 0.542
0.333 +/- 0.564
0.947 +/- 0.567
0.963 +/- 0.418
1.033 +/- 0.444
2.0 +/- 0.774
